In [ ]:
!git clone https://github.com/FabriDeCastelli/HLT-News-Category.git

import os
os.chdir("/content/HLT-News-Category/src/test")

In [1]:
import warnings
import os
import sys

warnings.filterwarnings('ignore')
current_dir = %pwd

parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))
sys.path.append(parent_dir)

In [2]:
from src.main.models.bidirLSMT import BidirectionalLSTM
from src.main.pipeline.functions import clean_text, stop_words_removal, tokenize
from src.main.utilities.utils import get_dataset, create_embedding_matrix, load_pretrained_embedddings
from sklearn.model_selection import train_test_split
from config.config import TENSORBOARD_LOGS, EMBEDDINGS_PATH, glove_file, google_file, fastText_file, VOCAB_SIZE, embedding_matrix
import keras as K

import warnings
warnings.filterwarnings("ignore")

In [3]:
bidirLSTM = BidirectionalLSTM()

bidirLSTM.pipeline = [clean_text, stop_words_removal, tokenize]

inputs, targets = get_dataset(one_hot=True)
inputs = bidirLSTM.run_pipeline(inputs, save=False)

VOCAB_SIZE = inputs.shape[0] + 1

pretrained_embeddings = load_pretrained_embedddings(os.path.join(EMBEDDINGS_PATH, glove_file), "glove")
find_ratio, unmached_words = create_embedding_matrix(pretrained_embeddings)

# stratified train test split
x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2, random_state=42, stratify=targets)

Pipeline execution time: 0:00:12.002694


In [4]:
import random

print("Find ratio:", find_ratio)
print("Unmached tokens count:", len(unmached_words))
print("Unmached words examples:\n" + str(random.sample(unmached_words, 10)))
print(str(random.sample(unmached_words, 10)))

Find ratio: 0.8738624620820694
Unmached tokens count: 3784
Unmached words examples:
["gap'", "kong's", "runner'", "graham's", "knight'", "i'm", "eve'", 'epa’s', "'champion", "'masterchef"]
["tonight'", "worker's", "'texas", 'monday’s', "soul's", "supremacist'", "candidates'", 'skyscanner', "supporters'", "devil's"]


In [8]:
callbacks = [
    K.callbacks.EarlyStopping(monitor='val_loss', patience=2),
    K.callbacks.TensorBoard(TENSORBOARD_LOGS.format(repr(bidirLSTM)))
]
bidirLSTM.grid_search(x_train, y_train, callbacks=callbacks, validation_split=0.15, n_iter=1, refit=True)

Trial 1 Complete [00h 14m 45s]
val_accuracy: 0.7633382081985474

Best val_accuracy So Far: 0.7633382081985474
Total elapsed time: 00h 14m 45s
1024/2301 ━━━━━━━━━━━━━━━━━━━━ 3:09 148ms/step - accuracy: 0.4563 - loss: 1.3497

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "{TENSORBOARD_LOGS.format(repr(bidirLSTM))}" --host localhost

In [ ]:
bidirLSTM.evaluate(x_test, y_test)

In [ ]:
bidirLSTM.save_model()

In [ ]:
a = BidirectionalLSTM.load_model()
a.evaluate(x_test, y_test)